In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import copy
import os

In [2]:
import pandas as pd
import numpy as np

In [4]:
import sys
# We need to include the path to pyrankability. This could be different for Tim, but altneratively he could point to your copy
sys.path.insert(0,"/disk/home/amy/rankability_toolbox_dev")

In [9]:
import pyrankability

In [5]:
sys.path.insert(0,"/disk/home/amy/sensitivity_study/src")

In [27]:
from sensitivity_tests import *
from utilities import *

In [14]:
from base import *

In [7]:
games={}
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]
for year in years:
    games[year] = read_data('../data/%steams.txt'%year,'../data/%sgames.txt'%year,'../data/%sMadnessTeams.txt'%year)
games[year]

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
0,1,16,737011,2017-11-13,1,83,-1,69,Arkansas_St,Abilene_Chr,0,0
1,1,41,737114,2018-02-24,-1,74,1,72,Cent_Arkansas,Abilene_Chr,0,0
3,1,143,737018,2017-11-20,-1,75,1,67,Lipscomb,Abilene_Chr,1,0
4,1,143,737045,2017-12-17,1,67,-1,65,Lipscomb,Abilene_Chr,1,0
5,1,199,737056,2017-12-28,1,77,-1,74,New_Orleans,Abilene_Chr,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5535,351,318,737048,2017-12-20,1,91,-1,74,Utah_St,Youngstown_St,0,0
5536,351,339,737086,2018-01-27,1,85,-1,67,WI_Green_Bay,Youngstown_St,0,0
5537,351,340,737084,2018-01-25,1,66,-1,55,WI_Milwaukee,Youngstown_St,0,0
5538,351,347,737074,2018-01-15,1,77,-1,67,Wright_St,Youngstown_St,1,0


In [8]:
year = "2002"
games[year]

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
0,1,34,731264,2002-02-18,1,67,-1,51,BYU,Air_Force,0,0
1,1,54,731241,2002-01-26,1,60,-1,57,Colorado_St,Air_Force,0,0
2,1,54,731271,2002-02-25,-1,59,1,51,Colorado_St,Air_Force,0,0
3,1,68,731196,2001-12-12,1,63,-1,53,Denver,Air_Force,0,0
4,1,91,731185,2001-12-01,1,68,-1,52,Fresno_St,Air_Force,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4785,327,315,731243,2002-01-28,1,53,-1,50,West_Virginia,Youngstown_St,0,0
4786,327,315,731275,2002-03-01,0,86,0,65,West_Virginia,Youngstown_St,0,0
4787,327,316,731217,2002-01-02,-1,77,1,55,WI_Green_Bay,Youngstown_St,0,0
4788,327,316,731255,2002-02-09,1,96,-1,76,WI_Green_Bay,Youngstown_St,0,0


In [16]:
# TODO: We should really spot check the frac part to make sure data is right
# Note to future self: Parameters from FODS paper but might need to be optimized
direct_thres = 2
spread_thres = 2
weight_indirect = 0.5
Ds = {}
fracs = [0.5, 0.75, 1.]
for year in games.keys():
    print(year)
    Ds[year] = {}
    madness_teams = np.unique(list(games[year].team1_name.loc[games[year].team1_madness == 1]) + list(games[year].team2_name.loc[games[year].team2_madness == 1]))
    game_list = list(games[year].index)
    
    game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                            "team1_score":games[year]['points1'],
                            "team1_H_A_N": games[year]['H_A_N1'],
                            "team2_name":games[year]['team2_name'],
                            "team2_score":games[year]['points2'],
                            "team2_H_A_N": games[year]['H_A_N1'],
                            "date": games[year]['date']
                           }).sort_values(by='date').drop('date',axis=1)
    for frac in fracs:
        upper = int(len(game_df)*frac)
        game_df_sample = game_df.iloc[:upper,:]
        map_func = lambda linked: support_map_vectorized_direct_indirect_weighted(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
        Ds[year][frac] = pyrankability.construct.V_count_vectorized(game_df_sample,map_func).loc[madness_teams,madness_teams]

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [28]:
pairs = [(0.5,0.75),(0.75,1.)]
rankings = {}
for year in games.keys():
    rankings[year] = []
    for pair in pairs:
        D1 = Ds[year][pair[0]]
        D2 = Ds[year][pair[1]]
        ranking1 = MasseyRankingAlgorithm().rank(D1.fillna(0).values)
        ranking2 = MasseyRankingAlgorithm().rank(D2.fillna(0).values)
        rankings[year].append((ranking1,ranking2))

In [30]:
taus = {}
for year in games.keys():
    taus[year] = {}
    for i in range(len(pairs)):
        pair = pairs[i]
        ranking1, ranking2 = rankings[year][i]
        taus[year][pair] = kendall_tau(ranking1,ranking2)

In [31]:
taus

{'2002': {(0.5, 0.75): 0.7000000000000001, (0.75, 1.0): 0.7701923076923077},
 '2003': {(0.5, 0.75): 0.6001984126984127, (0.75, 1.0): 0.7857142857142857},
 '2004': {(0.5, 0.75): 0.5182692307692308, (0.75, 1.0): 0.8105769230769231},
 '2005': {(0.5, 0.75): 0.6153846153846155, (0.75, 1.0): 0.8250000000000001},
 '2006': {(0.5, 0.75): 0.4634615384615385, (0.75, 1.0): 0.7682692307692308},
 '2007': {(0.5, 0.75): 0.7028846153846154, (0.75, 1.0): 0.8173076923076924},
 '2008': {(0.5, 0.75): 0.6309523809523809, (0.75, 1.0): 0.7638888888888887},
 '2009': {(0.5, 0.75): 0.5714285714285714, (0.75, 1.0): 0.8303571428571428},
 '2010': {(0.5, 0.75): 0.5567307692307693, (0.75, 1.0): 0.7855769230769231},
 '2011': {(0.5, 0.75): 0.6014047410008778, (0.75, 1.0): 0.8129938542581211},
 '2012': {(0.5, 0.75): 0.6047037539574853, (0.75, 1.0): 0.8263229308005429},
 '2013': {(0.5, 0.75): 0.5223880597014926, (0.75, 1.0): 0.717774762550882},
 '2014': {(0.5, 0.75): 0.5338015803336259, (0.75, 1.0): 0.7919227392449516},


In [39]:
results = pd.DataFrame(columns=pairs+["Year"]).set_index("Year")
for year in games.keys():
    data = [taus[year][pair] for pair in pairs]
    results = results.append(pd.Series(data,index=results.columns,name=year))
results

,"(0.5, 0.75)","(0.75, 1.0)"
Year,,
2002,0.700000,0.770192
2003,0.600198,0.785714
2004,0.518269,0.810577
2005,0.615385,0.825000
2006,0.463462,0.768269
2007,0.702885,0.817308
2008,0.630952,0.763889
2009,0.571429,0.830357
2010,0.556731,0.785577


In [71]:
df_ks = []
df_years = []
df_fracs = []
df_details = []
for year in games.keys():
    print(year)
    for pair in pairs:
        D = Ds[year][pair[0]]
        #C = pyrankability.construct.C_count(D,0)
        k,details = pyrankability.rank.solve(D.fillna(0),method='lop',lazy=False, cont=True)
        df_ks.append(k)
        df_years.append(year)
        df_fracs.append(pair[0])
        df_details.append(details)

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [72]:
rankability_results = pd.DataFrame({"k":df_ks,"Year":df_years,"Fraction":df_fracs})
rankability_results

,k,Year,Fraction
0,336.500000,2002,0.50
1,642.357143,2002,0.75
2,287.500000,2003,0.50
3,540.900001,2003,0.75
4,313.000000,2004,0.50
5,588.000000,2004,0.75
6,293.500000,2005,0.50
7,588.000000,2005,0.75
8,318.000000,2006,0.50
9,585.000000,2006,0.75


In [56]:
for_corr = rankability_results.loc[rankability_results.Fraction==0.5].set_index('Year').join(results)

In [57]:
for_corr.corr()

,k,Fraction,"(0.5, 0.75)","(0.75, 1.0)"
k,1.000000,NaN,0.332619,-0.042454
Fraction,NaN,NaN,NaN,NaN
"(0.5, 0.75)",0.332619,NaN,1.000000,0.348263
"(0.75, 1.0)",-0.042454,NaN,0.348263,1.000000


In [58]:
from scipy.stats import pearsonr 

In [59]:
pearsonr(for_corr['k'],for_corr[(0.5,0.75)])

(0.3326188359935908, 0.19206924851493423)

In [88]:
from scipy.stats import skew

all_score_df = pd.DataFrame(columns=['Year','# X* frac','k','# X* frac top 40','Pair'])
c=0
for year in games.keys():
    print(year)
    for pair in pairs:
        V = Ds[year][pair[0]]
        rresults = rankability_results.iloc[c,:]
        k = rresults['k']
        details = df_details[c]
        x = pd.DataFrame(details['x'],index=V.index,columns=V.columns)
        r = x.sum(axis=0)
        order = np.argsort(r)
        xstar = x.iloc[order,:].iloc[:,order]
        xstar.loc[:,:] = pyrankability.common.threshold_x(xstar.values)
        inxs = np.triu_indices(len(xstar),k=1)
        xstar_upper = xstar.values[inxs[0],inxs[1]]
        nfrac_upper = sum((xstar_upper > 0) & (xstar_upper < 1))
        flat_frac = ((xstar > 0) & (xstar < 1)).sum(axis=0)
        nfrac_top_40 = flat_frac.iloc[:40].sum()
        entry = pd.Series([year,nfrac_upper*2,k,nfrac_top_40,pair],all_score_df.columns,name=c)
        c+=1
        all_score_df = all_score_df.append(entry)
all_score_df.set_index("Year",inplace=True)

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [91]:
all_score_df = all_score_df.loc[all_score_df.Pair == (0.5,0.75)].drop('Pair',axis=1).join(results)

In [94]:
import sklearn.linear_model as skl_lm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV

def evaluate(df,pred_col=(0.5,0.75),feature_cols=["k",'# X* frac'],model=DummyRegressor(),param_grid={}):
    # fill in evaluat
    loo = LeaveOneOut()
    y = df[pred_col]
    X = df[feature_cols]
    
    grid = GridSearchCV(model,param_grid,refit=True,verbose=2,n_jobs=-1)
    scores = cross_val_score(grid, X, y, scoring="neg_mean_absolute_error", cv=loo, n_jobs=1)
    return pd.Series([len(scores),np.mean(np.abs(scores)),np.std(scores)],index=["Folds","MAE","STD"])

In [95]:
evaluate(all_score_df)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.0s finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.5s finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.5s finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric res

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric res

Folds    17.000000
MAE       0.057941
STD       0.041575
dtype: float64

In [ ]:
evaluate(all_score_df,model=SVR(gamma='scale'),param_grid = {'C': [0.1,1,10,100], 'epsilon': [0.1,0.5,1],'kernel': ['linear']})

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  2.9min finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  4.0min finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  36 | elapsed:    0.1s remaining:    0.1s


Fitting 3 folds for each of 12 candidates, totalling 36 fits


...

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.1s finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  1.5min finished
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/data/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits
